# Wild deserts image classification
Training is done in [Google Colab](https://colab.research.google.com/drive/1SOflnQu87P5v9klSefdetHO2b85fqTIR?usp=sharing)

### Import libraries

In [23]:
from ultralytics import YOLO
import os
import pandas as pd
import time
from glob import glob
import re
import numpy as np
from skmultilearn.model_selection import iterative_train_test_split
import shutil




#### Sort images into day/night

##### Testing

In [ ]:
import cv2 # computer vision library
import helpers # read input in

import numpy as np 
import matplotlib.pyplot as plt # For display images
import matplotlib.image as mpimgimport # computer vision library
import helpers # read input in

import numpy as np 
import matplotlib.pyplot as plt # For display images
import matplotlib.image as mpimg

image_dir_training_day = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\training\\day_night\\day\\train"
image_dir_test_day = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\training\\day_night\\day\\test"
image_dir_training_night = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\training\\day_night\\night\\train"
image_dir_test_night = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\training\\day_night\\night\\test"

import cv2
import os
import numpy as np

def classify_image(image_path):
    image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    brightness = hsv_image[:, :, 2].mean()  # V channel represents brightness
    return 'day' if brightness > 80 else 'night'  # You can adjust the threshold as needed

def classify_images_in_directory(directory):
    classifications = {'day': [], 'night': []}
    for filename in os.listdir(directory):
        if filename.endswith(".JPG") or filename.endswith(".jpg"):
            image_path = os.path.join(directory, filename)
            classification = classify_image(image_path)
            classifications[classification].append(filename)
    return classifications

# Classify images in each directory
training_day_classifications = classify_images_in_directory(image_dir_training_day)
training_night_classifications = classify_images_in_directory(image_dir_training_night)
test_day_classifications = classify_images_in_directory(image_dir_test_day)
test_night_classifications = classify_images_in_directory(image_dir_test_night)

print("Training Day Classifications:", training_day_classifications)
print("Training Night Classifications:", training_night_classifications)
print("Test Day Classifications:", test_day_classifications)
print("Test Night Classifications:", test_night_classifications)

##### Actual classification of datasets

In [60]:
import os
import shutil
import cv2

# Define the source directory and the target directories for day and night images
image_source_directory = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\training\\model_training_no_split\\14_classes_b_plus_empty\\images"
label_source_directory = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\training\\model_training_no_split\\14_classes_b_plus_empty\\labels"

day_directory_images = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\training\\model_training_day\\images"
day_directory_labels = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\training\\model_training_day\\labels"

night_directory_images = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\training\\model_training_night\\images"
night_directory_labels = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\training\\model_training_night\\labels"

# Create the target directories if they do not exist
os.makedirs(day_directory_images, exist_ok=True)
os.makedirs(day_directory_labels, exist_ok=True)
os.makedirs(night_directory_images, exist_ok=True)
os.makedirs(night_directory_labels, exist_ok=True)

def classify_image(image_path):
    image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    brightness = hsv_image[:, :, 2].mean()  # V channel represents brightness
    return 'day' if brightness > 80 else 'night'  # You can adjust the threshold as needed

# Classify and copy images and labels
for filename in os.listdir(image_source_directory):
    if filename.endswith(".JPG") or filename.endswith(".jpg"):
        image_path = os.path.join(image_source_directory, filename)
        label_path = os.path.join(label_source_directory, filename.replace(".JPG", ".txt").replace(".jpg", ".txt"))
        classification = classify_image(image_path)
        if classification == 'day':
            shutil.copy(image_path, os.path.join(day_directory_images, filename))
            shutil.copy(label_path, os.path.join(day_directory_labels, os.path.basename(label_path)))
        else:
            shutil.copy(image_path, os.path.join(night_directory_images, filename))
            shutil.copy(label_path, os.path.join(night_directory_labels, os.path.basename(label_path)))

print("Classification and copying completed!")

Classification and copying completed!


In [54]:
help(os.makedirs)

Help on function makedirs in module os:

makedirs(name, mode=511, exist_ok=False)
    makedirs(name [, mode=0o777][, exist_ok=False])

    Super-mkdir; create a leaf directory and all intermediate ones.  Works like
    mkdir, except that any intermediate path segment (not just the rightmost)
    will be created if it does not exist. If the target directory already
    exists, raise an OSError if exist_ok is False. Otherwise no exception is
    raised.  This is recursive.



### Reclassify labels
- This uses some basic text editing to replace values that are being reclassified
- Faster than reclassifying using [LabelStudio](https://labelstud.io/)
    - Can use LabelStudio if needed. Install using Docker is easiest
- I also split images into new test/train/val sets after reclassifying.
    - I use iterative_train_test_split from Scikit-multilearn to perform stratified classification
        - Need to stratify because of the imbalanced dataset


#### Reclassify images

In [68]:
for file_path in glob('C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\training\\model_training_day\\labels\\*.txt'):
    print(f'{file_path}')
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        modified_lines = []
        for line in lines:
            to_remove = re.match(r'^(0 .*\n|3 .*\n|4 .*\n|7 .*\n|9 .*\n|10 .*\n|13 .*\n|15 .*\n|6 .*\n)', line)
            kangaroo = re.match(r'^(14 |12 |8 |5 )', line)
            rabbit = re.match(r'^(11 )', line)
            dingo = re.match(r'^(2 )', line)
            if to_remove:
                continue  # Skip appending this line
            elif kangaroo:
                modified_line = re.sub(r'^(14 |12 |8 |5 )', '0 ', line)
            elif rabbit:
                modified_line = re.sub(r'^(11 )', "2 ", line)
            elif dingo:
                modified_line = re.sub(r'^(2 )', "3 ", line)    
            else:
                modified_line = line  
            
            modified_lines.append(modified_line)
    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(modified_lines)
        print(f'File written to: {file_path}')

print("Done!")

C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\training\model_training_day\labels\2020-06-04 07-59-46 M 1_3.txt
File written to: C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\training\model_training_day\labels\2020-06-04 07-59-46 M 1_3.txt
C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\training\model_training_day\labels\2020-06-04 08-00-15 M 2_3.txt
File written to: C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\training\model_training_day\labels\2020-06-04 08-00-15 M 2_3.txt
C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\training\model_training_day\labels\2020-06-08 08-21-40 M 2_3.txt
File written to: C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\training\model_training_day\labels\2020-06-08 08-21-40 M 2_3.txt
C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild 

#### Split the reclassified images

This block actually just extracts the data from the YOLO labels and puts it into two numpy arrays

In [69]:

# Path to your YOLO .txt annotations
annotations_path = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\training\\model_training_day\\labels\\*.txt"

# List all .txt files
txt_files = glob(annotations_path)

X = []  # Will store image paths or anything representing the "features"
y = []  # Will store the multi-label vectors of shape (16,)

for txt_file in txt_files:
    # Construct the corresponding image path
    # (assuming .jpg files, adjust if your images are .png, etc.)
    img_file = txt_file.replace(".txt", ".JPG")
    img_file = img_file.replace("labels", "images")
    # Initialize a 4-dimensional zero vector for the labels !!! MAKE SURE TO CHANGE IF YOU HAVE MORE CLASSES !!!
    labels = np.zeros(4, dtype=int)
    print(txt_file)
    # Read the YOLO annotation file
    with open(txt_file, "r") as f:
        for line in f:
            items = line.strip().split()
            if len(items) >= 5:
                class_id = int(items[0])
                # Mark that class_id as present
                labels[class_id] = 1

    # Append to X and y
    X.append(img_file)     # or store actual image data if needed
    y.append(labels)

X = np.array(X)
X = X.reshape(-1, 1)
y = np.array(y)


C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\training\model_training_day\labels\2020-06-04 07-59-46 M 1_3.txt
C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\training\model_training_day\labels\2020-06-04 08-00-15 M 2_3.txt
C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\training\model_training_day\labels\2020-06-08 08-21-40 M 2_3.txt
C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\training\model_training_day\labels\2020-06-17 06-55-02 M 1_3.txt
C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\training\model_training_day\labels\2020-07-16 16-23-11 M 2_3.txt
C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\training\model_training_day\labels\2020-07-16 16-25-33 M 3_3.txt
C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\training\model_trainin

This block is the actual splitting

In [70]:

test_size = 0.1 #10% of the data will be used for test

# Perform the iterative train/test split
X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size=test_size)
X_test = X_test.ravel()  
X_train = X_train.ravel()
print("After first split:")
print("  train:", X_train.shape, y_train.shape)

print("  test:     ", X_test.shape,    y_test.shape)

After first split:
  train: (1363,) (1363, 4)
  test:      (152,) (152, 4)


Validate image split

In [71]:
import numpy as np

def count_classes(y_data):
    """ Sum over axis=0 to get how many samples contain each class. """
    return y_data.sum(axis=0)

def print_class_stats(name, y_data):
    counts = count_classes(y_data)
    print(f"{name} - counts per class: {counts}")
    print(f"{name} - total samples: {len(y_data)}\n")

print_class_stats("TRAIN", y_train)
print_class_stats("TEST",  y_test)



TRAIN - counts per class: [228  99 106  36]
TRAIN - total samples: 1363

TEST - counts per class: [25 11 12  4]
TEST - total samples: 152



Move the files into their actual folders

In [72]:
import shutil
from tqdm import tqdm
#Change these to your directories
labels_dir = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\training\\model_training_day\\labels"
images_dir = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\training\\model_training_day\\images"
# Create "labels" and "images" folders in each directory
for directory in [labels_dir, images_dir]:
    os.makedirs(os.path.join(directory, "test"), exist_ok=True)
    os.makedirs(os.path.join(directory, "train"), exist_ok=True)
    #os.makedirs(os.path.join(directory, "val"), exist_ok=True)
# Copy test images and labels. Note that this leaves the files in the original directories too
for img_path in tqdm(X_test, desc="Copying test data"):
    
    txt_path = img_path.replace(".JPG", ".txt")
    txt_path = txt_path.replace("images", "labels")

    shutil.copy(img_path, os.path.join(images_dir, "test"))
    shutil.copy(txt_path, os.path.join(labels_dir, "test"))
# Copy training images and labels
for img_path in tqdm(X_train, desc="Copying training data"):
    txt_path = img_path.replace(".JPG", ".txt")  # or .png, whichever you have
    txt_path = txt_path.replace("images", "labels")
    shutil.copy(img_path, os.path.join(images_dir, "train"))
    shutil.copy(txt_path, os.path.join(labels_dir, "train"))

# Copy validation images and labels
'''for img_path in tqdm(X_val, desc="Copying validation data"):
    txt_path = img_path.replace(".JPG", ".txt")  # or .png, whichever you have
    txt_path = txt_path.replace("images", "labels")

    shutil.copy(img_path, os.path.join(images_dir, "val"))
    shutil.copy(txt_path, os.path.join(labels_dir, "val"))'''




Copying training data: 100%|██████████| 1363/1363 [00:31<00:00, 43.05it/s]


'for img_path in tqdm(X_val, desc="Copying validation data"):\n    txt_path = img_path.replace(".JPG", ".txt")  # or .png, whichever you have\n    txt_path = txt_path.replace("images", "labels")\n\n    shutil.copy(img_path, os.path.join(images_dir, "val"))\n    shutil.copy(txt_path, os.path.join(labels_dir, "val"))'

### Set variables/filepaths 

In [24]:
validation_root_directory = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\validation"
model = YOLO("C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\training_output\\model_training_proper_split_v1_120125\\output\\content\\runs\\detect\\train6\\weights\\best.pt")

### Trained model validation/prediction
The model needs to be trained on Colab prior to this step

##### This block is an old training workflow but I have kept it here for reference

In [19]:
import shutil
all_detections = []
for folder in os.listdir(f"{validation_root_directory}/all_images/"):
    print(folder)
    directory = f"{validation_root_directory}/all_images/{folder}/"
    
    for counter, image in enumerate(os.listdir(directory), start=1):
        name, ext = os.path.splitext(image)
        print(f"{counter}/{len(os.listdir(directory))}")
        
        if ext == '.JPG':  
            
            predictions = model.predict(
                source=os.path.join(directory, image),
                save=True,
                save_txt=True,
                save_conf=False,
                imgsz=640,
                conf=0.1,
                iou=0.5,
                augment=True,
                project=f"{validation_root_directory}/output3/labels_images/predict",
                name = "test"
            )
            os.makedirs(f"{validation_root_directory}/output3/labels_images/predict/images", exist_ok=True)
            shutil.copy(os.path.join(directory, image), f"{validation_root_directory}/output3/labels_images/predict/images/{name}.jpg")
            if predictions[0]:
                species = model.names[int(predictions[0].boxes.cls[0])]
                conf = round(float(predictions[0].boxes.conf[0]), 3)
                path = predictions[0].path
                print(path)
                bbox = predictions[0].boxes.xywh.tolist()
                df = pd.DataFrame({'path': path, 'species': species, 'confidence': conf, 'bbox': bbox, })
                all_detections.append(df)
                newpath = f"{validation_root_directory}/output3/{species}"
            else:
                print("No detections were made.")
                newpath = f"{validation_root_directory}/output3/empty"
            
            os.makedirs(newpath, exist_ok=True)
            predictions[0].save(f"{newpath}/{name}_{conf if predictions[0] else ''}.jpg")
            
            os.makedirs(all_images_path, exist_ok=True)
            predictions[0].save(f"{all_images_path}/{name}_{conf if predictions[0] else ''}_{species if predictions[0] else 'empty'}.jpg")

timestamp = time.strftime("%Y%m%d_%H%M%S")
if all_detections:
    final_df = pd.concat(all_detections)
    final_df.to_csv(f"{validation_root_directory}/output3/detections_{timestamp}.csv", index=False)
else:
    print("No detections were made.")


FC2
1/72

image 1/1 C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\validation\all_images\FC2\2017-11-18 11-05-30 M 1_3.JPG: 384x640 (no detections), 250.9ms
Speed: 2.0ms preprocess, 250.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\validation\output3\labels_images\predict\test
0 label saved to C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\validation\output3\labels_images\predict\test\labels
No detections were made.
2/72

image 1/1 C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\validation\all_images\FC2\2017-11-26 23-09-13 M 1_3.JPG: 384x640 1 Kangaroo, 366.8ms
Speed: 2.0ms preprocess, 366.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classificatio

KeyboardInterrupt: 

##### New training block

Need to split into day/night images for the day/night models

In [2]:
import os
import shutil
import cv2

# Define the source directory and the target directories for day and night images
image_source_directory = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\validation\\all_images"

day_directory_images = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\validation\\all_images_day_night\\day"

night_directory_images = "C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\validation\\all_images_day_night\\night"

# Create the target directories if they do not exist
os.makedirs(day_directory_images, exist_ok=True)
os.makedirs(night_directory_images, exist_ok=True)

def classify_image(image_path):
    image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    brightness = hsv_image[:, :, 2].mean()  # V channel represents brightness
    return 'day' if brightness > 80 else 'night'  # You can adjust the threshold as needed

# Classify and copy images and labels

for subdir, _, files in os.walk(image_source_directory):
    for filename in files:
        if filename.endswith(".JPG") or filename.endswith(".jpg"):
            image_path = os.path.join(subdir, filename)
            classification = classify_image(image_path)
            if classification == 'day':
                shutil.copy(image_path, os.path.join(day_directory_images, filename))
            else:
                shutil.copy(image_path, os.path.join(night_directory_images, filename))
    

print("Classification and copying completed!")

error: OpenCV(4.10.0) D:\bld\libopencv_1735821196820\work\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [25]:
main_directory = f"{validation_root_directory}\\all_images"
all_predictions = []
for folder in os.listdir(main_directory):
    directory = f"{main_directory}\\{folder}"
    predictions = model.predict(
        source=os.path.join(directory),
        save=False, # I don't save anything here because I want to sort into folders
        save_txt=False,
        save_conf=False,
        imgsz=1280,
        conf=0.1,
        iou=0.5,
        augment=True,
        project=f"{validation_root_directory}/output3/labels_images/predict",
        show=False)
    all_predictions = all_predictions + predictions



image 1/70 C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\validation\all_images\FC2\2017-11-18 11-05-30 M 1_3.JPG: 736x1280 1 Kangaroo, 652.8ms
image 2/70 C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\validation\all_images\FC2\2017-11-26 23-09-13 M 1_3.JPG: 736x1280 1 Kangaroo, 590.6ms
image 3/70 C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\validation\all_images\FC2\2017-12-01 18-00-58 M 1_3.JPG: 736x1280 1 Kangaroo, 1 Cat, 569.9ms
image 4/70 C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\validation\all_images\FC2\2017-12-01 20-58-46 M 3_3.JPG: 736x1280 2 Cats, 2 Rabbits, 487.4ms
image 5/70 C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image classification\validation\all_images\FC2\2017-12-01 21-01-50 M 3_3.JPG: 736x1280 1 Kangaroo, 486.2ms
image 6/70 C:\Users\willo\OneDrive - UNSW\Documents\Work\CES\Wild Deserts\Image 

This block is if you want to do validation in digikam or R later

In [28]:
all_detections = []

for i in range(len(all_predictions)):
    #if detections
    if all_predictions[i].boxes:  # Check if there are any boxes in the prediction
        species = model.names[int(all_predictions[i].boxes.cls[0])]
        conf = round(float(all_predictions[i].boxes.conf[0]), 3)
        path = all_predictions[i].path
        # Use the correct index i instead of 1
        name = os.path.splitext(os.path.basename(all_predictions[i].path))[0]
        bbox = all_predictions[i].boxes.xywh.tolist()
        # 5) Save the label file in the "labels" folder
        labels_dir = f"{validation_root_directory}/output3/labels_images/predict/labels"
        os.makedirs(labels_dir, exist_ok=True)
        labels_path = f"{labels_dir}/{name}.txt"
        all_predictions[i].save_txt(labels_path)
    #if no detections
    else:
        species = "none"
        conf = "none"
        path = all_predictions[i].path
        name = os.path.splitext(os.path.basename(all_predictions[i].path))[0]
        bbox = "none"
        labels_dir = f"{validation_root_directory}/output3/labels_images/predict/labels"
        os.makedirs(labels_dir, exist_ok=True)
        labels_path = f"{labels_dir}/{name}.txt"
        with open(labels_path, 'w') as f:
            f.write('')
        print(f"No detections for prediction {i}")
        
    # 1) Create a folder for this species
    newpath = f"{validation_root_directory}/output3/{species}"
    os.makedirs(newpath, exist_ok=True)

    # 2) Save the annotated image to the species folder
    all_predictions[i].save(f"{newpath}/{name}.jpg")

    # 3) Save the original image in the "images" folder
    images_dir = f"{validation_root_directory}/output3/labels_images/predict/images"
    os.makedirs(images_dir, exist_ok=True)
    shutil.copy(path, images_dir)
    path_original = f"{images_dir}/{name}.JPG"

    # 4) Save the annotated image in the "annotated" folder
    annotated_dir = f"{validation_root_directory}/output3/labels_images/predict/annotated"
    os.makedirs(annotated_dir, exist_ok=True)
    path_annotated = f"{annotated_dir}/{name}.JPG"
    all_predictions[i].save(path_annotated)



    # 5) Build a DataFrame row (no classification check here)
    df = pd.DataFrame({
        'path_original': [path_original],
        'species': [species],
        'confidence': [conf],
        'bbox': [bbox],
        'path_annotated': [path_annotated],
        'label_path': [labels_path]
    })

    # 6) Append this to all_detections
    all_detections.append(df)
    


# Finally, save all detections to a CSV
timestamp = time.strftime("%Y%m%d_%H%M%S")
if all_detections:
    final_df = pd.concat(all_detections, ignore_index=True)
    final_df.to_csv(f"{validation_root_directory}/output3/detections_{timestamp}.csv", index=False)


No detections for prediction 10
No detections for prediction 96
No detections for prediction 111
No detections for prediction 141
No detections for prediction 169
No detections for prediction 184
No detections for prediction 210
No detections for prediction 220
No detections for prediction 243
No detections for prediction 249
No detections for prediction 278
No detections for prediction 279
No detections for prediction 324
No detections for prediction 336
No detections for prediction 340
No detections for prediction 356
No detections for prediction 371
No detections for prediction 375
No detections for prediction 415
No detections for prediction 416
No detections for prediction 450
No detections for prediction 562
No detections for prediction 636
No detections for prediction 647
No detections for prediction 764
No detections for prediction 878
No detections for prediction 957
No detections for prediction 1022
No detections for prediction 1023
No detections for prediction 1024
No detect

73

In [ ]:
import cv2
import matplotlib.pyplot as plt

for i in range(len(all_predictions)):
    if all_predictions[i].boxes:  # Check if there are any boxes in the prediction
        species = model.names[int(all_predictions[i].boxes.cls[0])]
        conf = round(float(all_predictions[i].boxes.conf[0]), 3)
        path = all_predictions[i].path
        name = os.path.splitext(os.path.basename(all_predictions[i].path))[0]
        bbox = all_predictions[i].boxes.xywh.tolist()
        # save the label file to the labels folder
        labels_path = f"{validation_root_directory}/output3/labels_images/predict/labels/{name}.txt"
        all_predictions[i].save_txt(labels_path)

    else:
        species = "none"
        conf = "none"
        path = all_predictions[i].path
        name = os.path.splitext(os.path.basename(all_predictions[i].path))[0]
        bbox = "none"
        labels_dir = f"{validation_root_directory}/output3/labels_images/predict/labels"
        os.makedirs(labels_dir, exist_ok=True)
        labels_path = f"{labels_dir}/{name}.txt"
        with open(labels_path, 'w') as f:
            f.write('')
        print(f"No detections for prediction {i}")
    
    # save the annotated image to the species folder
    newpath = f"{validation_root_directory}/output3/{species}"  
    os.makedirs(newpath, exist_ok=True)
    all_predictions[i].save(f"{newpath}/{name}.jpg")
        
    # save the original image to the images folder
    os.makedirs(f"{validation_root_directory}/output3/labels_images/predict/images", exist_ok=True)
    shutil.copy(path, f"{validation_root_directory}/output3/labels_images/predict/images")
    path_original = f"{validation_root_directory}/output3/labels_images/predict/images/{name}.JPG"
        
    # save the annotated image to the annotated folder
    path_annotated = f"{validation_root_directory}/output3/labels_images/predict/annotated/{name}.JPG"
    os.makedirs(f"{validation_root_directory}/output3/labels_images/predict/annotated", exist_ok=True)
    all_predictions[i].save(f"{validation_root_directory}/output3/labels_images/predict/annotated/{name}.JPG")
        
    
        
    # Display the annotated image in a pop-up window with smaller size
    img = cv2.imread(path_annotated)
    img_resized = cv2.resize(img, (800, 600))  # Resize the image to 800x600
    cv2.imshow(f"Species: {species}, Confidence: {conf}", img_resized)

    cv2.moveWindow(f"Species: {species}, Confidence: {conf}", 0, 0)  # Move the window to the top left corner
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # Prompt user for input
    result = input("Is the detection correct? (y/n): ").strip().lower()
    
    # save the data to a dataframe
    df = pd.DataFrame({'path_original': [path_original], 
                        'species': [species], 
                        'confidence': [conf], 
                        'bbox': [bbox], 
                        "path_annotated": [path_annotated],
                        "label_path": [labels_path],
                        "results": [result]})
    
    # append the dataframe to the list of all detections
    all_detections.append(df)
    

# Save the results to a CSV file
timestamp = time.strftime("%Y%m%d_%H%M%S")
if all_detections:
    final_df = pd.concat(all_detections)
    final_df.to_csv(f"{validation_root_directory}/output3/detections_{timestamp}.csv", index=False)


No detections for prediction 10
No detections for prediction 96
No detections for prediction 111
No detections for prediction 141


In [62]:
df

,path_original,species,confidence,bbox,path_annotated,label_path,results
0,C:\Users\willo\OneDrive - UNSW\Documents\Work\...,Rabbit,0.926,"[393.875732421875, 866.3226318359375, 156.3745...",C:\Users\willo\OneDrive - UNSW\Documents\Work\...,C:\Users\willo\OneDrive - UNSW\Documents\Work\...,y


In [54]:
predictions[1].path
name = os.path.splitext(os.path.basename(predictions[1].path))[0]
name


'2017-11-26 23-09-13 M 1_3'

In [27]:
predictions[0].

'C:\\Users\\willo\\OneDrive - UNSW\\Documents\\Work\\CES\\Wild Deserts\\Image classification\\validation\\output3\\labels_images\\predict\\test'